In [70]:
import requests
import datetime as dt            
import pandas as pd                      
import praw
import sys
sys.path.append("C:\\Users\\Tiziano Pacifico\\Desktop\\CodeTesi")
import credential
from datetime import datetime, timedelta
import numpy as np

# Note

Il primo passo è stato quello di estarre l'insieme delle submissions dall'api di pushshift in modo da avere una struttura dati snella di base contenente 3 informazioni:
SUBMISSION ID // UTC // UTC_STR
L'insieme delle submission ottenute tramite pushshift offre la maggior sicurezza di aver recuperato il maggior numero di esse attingendo ad un serbatoio che colleziona tutti i post di reddit.
Questo dataframe molto semplice verrà utilizato come base per tutte le future estrazioni.
Infatti l'api di reddit seppur più lentamente e con alcune restrizioni fornisce informazioni più dettagliate e aggiornate sui singoli oggetti.
Tecnicamente abbiamo inserito in un loop con un valore di 1000 iterazioni la chiamata alla api di pushshift. La chiamata è stata effettuata direttamente dall'api senza wrapper. Questo metodo in genere garantisce il migliore risultato in termini di tempo e di numero di dati trovati.
Ogni chiamata può ritornare al massimo 1000 elementi. Il numero di 1000 iterazioni su 1000 elementi da un valore massimo di 100000 di submission ritornate. LE submissions sono restituite in ordine cronologico inverso dalla più recente alla più datata. L'ultima submission ottenuta da una chiamata fornirà quindi una soglia di partenza per la prossima chiamata. 
Le submissions ottenute sono inferiori alle 100000 quindi il numero di 1000 iterazioni si è dimostrato molto superiore rispetto alle necessità.
Inoltre ogni singola chiamata non restituisce mai il numero di 1000 risposte. Il motivo di questo comportamento potrebbe essere dovuto al fallimento di alcuni thread di ricerca e potrebbe comportare la perdita di alcune submissions.
Il miss rate è comunque in genere inferiore al 5% e possiamo riterere il risultato ottenuto soddisfacente.
I dati estratti sono stati restituiti come lista di tuple. Ogni elemento della tupla è relativo ad una colonna del dataframe.
Il passaggio dalla lista al dataframe è stato effettuato iterativamente.
Il sistema di inserimento iterativo in un dataframe è però un processo lungo che rallenta proporzionalmente alle dimension del dataframe. Nei successivi passaggi si preferirà una soluzione differente che permette una maggiore effcienza.
Per via della modalita estrattiva il dataframe presenta alcuni duplicati. Infatti ogni nuova riciesta parte dalla data utc del messaggio più datato della richiesta precendente e aggiunge qualche secondo creando un piccolo accavallamento temporale che porta in alcuni casi a scaricare gli stessi messaggi.
I messaggi inizialmente scaricati sono stati 74942 mentre eliminando i duplicati sulla base dell'id della submission abbiamo ottenuto un dataset di 73943 elementi.

In [146]:
cols = ['id','utc','utc_datetime_str']

In [112]:
BASE_URL = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&before={}&size={}'
subreddit = 'ChatGPT'
before = datetime.now()
size = 1000

In [113]:
epoch = int(before.timestamp())

In [114]:
uri = BASE_URL.format(subreddit, epoch, size)

In [35]:
df = pd.DataFrame()

In [121]:
list_id_utc = []

In [122]:
for i in range(1000):
    res = (requests.get(uri)).json()['data']
    list_id_utc.extend(list(map(lambda post: (post['id'],post['created_utc'], post['utc_datetime_str']), res)))
    epoch = list_id_utc[-1][1] + 1
    uri =  BASE_URL.format(subreddit, epoch, size)

In [123]:
len(list_id_utc)

74942

# ----------------------------

In [160]:
df_id_utc = pd.DataFrame(columns=cols)

In [161]:
for submission in list_id_utc:
    row[cols[0]] = submission[0]
    row[cols[1]] = submission[1]
    row[cols[2]] = submission[2]
    df_id_utc.loc[len(df_id_utc)] = row

In [162]:
df_id_utc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74942 entries, 0 to 74941
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                74942 non-null  object
 1   utc               74942 non-null  int64 
 2   utc_datetime_str  74942 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.3+ MB


In [165]:
df_id_utc =  df_id_utc.drop_duplicates(subset='id', keep="first")

In [167]:
df_id_utc.to_json('C:\\Users\\Tiziano Pacifico\\Desktop\\TESI\\RedditDS\\id_utc.json', orient='records', lines=True)